In [4]:
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from model import ResNet
import numpy as np
import torchvision.datasets as dset
import torchvision.transforms as T

In [5]:
import copy
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader
from main import ChunkSampler

In [11]:
def train(loader_train, model, criterion, optimizer, gpu_dtype):
    model.train()
    for t, (X, y) in enumerate(loader_train):
        X_var = Variable(X.type(gpu_dtype))
        y_var = Variable(y.type(gpu_dtype)).long()

        scores = model(X_var)

        loss = criterion(scores, y_var)
        if (t+1) % 100 == 0:
            print('t = %d, loss = %.4f' % (t+1, loss.data))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [12]:
def check_accuracy(model, loader, gpu_dtype):
    num_correct = 0
    num_samples = 0
    model.eval()
    for X, y in loader:

        X_var = Variable(X.type(gpu_dtype), requires_grad=True)

        scores = model(X_var)
        _, preds = scores.data.cpu().max(1) 

        num_correct += (preds == y).sum()
        num_samples += preds.size(0)

    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [22]:
def struct_pruning(model, conv_amount, lin_amount, out=False):
    pruned_struct = copy.deepcopy(model)
    if out:
        dim = 0
    else:
        dim = 1
    for name, module in pruned_struct.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=conv_amount, n=2, dim=dim)
        elif isinstance(module, torch.nn.Linear):
            prune.ln_structured(module, name='weight', amount=lin_amount, n=2, dim=dim)
    return pruned_struct

In [23]:
def pruning_fine_tuning(model, conv_rate=0.5, fc_rate=0.5, epochs=20, filename='fine_tuned model', saving=False, out=False):
    criterion = nn.CrossEntropyLoss().cuda()
    gpu_dtype = torch.cuda.FloatTensor
    learning_rate = 1
    struct_model = struct_pruning(model, conv_rate, fc_rate, out)
    print('Model pruned with rates:', conv_rate, ',', fc_rate)
    struct_model = struct_model.type(gpu_dtype)
    optimizer = optim.SGD(struct_model.parameters(), lr=learning_rate,
                            momentum=0.9, weight_decay=0)
    for epoch in range(epochs):
        check_accuracy(struct_model, loader_train, gpu_dtype)
        print('Starting epoch %d' % (epoch + 1))
        train(loader_train, struct_model, criterion, optimizer, gpu_dtype)
        learning_rate *= 0.1 
    if saving:
        torch.save(struct_model, filename)
    return struct_model

In [15]:
global gpu_dtype
gpu_dtype = torch.cuda.FloatTensor
# define transforms for normalization and data augmentation
transform_augment = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(32, padding=4)])
transform_normalize = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
# get CIFAR-10 data
NUM_TRAIN = 45000
NUM_VAL = 5000

cifar10_train = dset.CIFAR10('./dataset', train=True, download=True,
                             transform=T.Compose([transform_augment, transform_normalize]))
loader_train = DataLoader(cifar10_train, batch_size=128,
                          sampler=ChunkSampler(NUM_TRAIN))
cifar10_val = dset.CIFAR10('./dataset', train=True, download=True,
                           transform=transform_normalize)
loader_val = DataLoader(cifar10_train, batch_size=128,
                        sampler=ChunkSampler(NUM_VAL, start=NUM_TRAIN))
cifar10_test = dset.CIFAR10('./dataset', train=False, download=True,
                            transform=transform_normalize)

loader_test = DataLoader(cifar10_test, batch_size=128)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [15]:
model_outputs = ResNet(n=5, res_option='A', use_dropout=True)
model_outputs.load_state_dict(torch.load('ResNet-34_20_epoch_outputs'))
model_outputs.eval()
#ResNet-34_20_epoch_outputs
#ResNet-34_20_epoch_weights

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (layers1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout2d(p=0.67, inplace=False)
      (relu2): ReLU(inplace=True)
    )
    (1): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), st

In [17]:
model_weights = ResNet(n=5, res_option='A', use_dropout=True)
model_weights.load_state_dict(torch.load('ResNet-34_20_epoch_weights'))
model_weights.eval()

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (layers1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout2d(p=0.67, inplace=True)
      (relu2): ReLU(inplace=True)
    )
    (1): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), str

In [20]:
outputs_prune_40 = pruning_fine_tuning(model_outputs, conv_rate=0.4, 
                                       fc_rate=0.4, epochs=200, 
                                       filename='model outputs prune 40', 
                                       saving=True)

Model pruned with rates: 0.4 , 0.4
Got 9968 / 45000 correct (22.15)
Starting epoch 1
t = 100, loss = 1.2626
t = 200, loss = 1.3669
t = 300, loss = 1.0989
Got 27174 / 45000 correct (60.39)
Starting epoch 2
t = 100, loss = 0.8654
t = 200, loss = 0.9709
t = 300, loss = 0.8769
Got 31722 / 45000 correct (70.49)
Starting epoch 3
t = 100, loss = 0.8722
t = 200, loss = 1.0537
t = 300, loss = 0.7051
Got 33116 / 45000 correct (73.59)
Starting epoch 4
t = 100, loss = 0.7139
t = 200, loss = 0.9215
t = 300, loss = 0.6608
Got 33499 / 45000 correct (74.44)
Starting epoch 5
t = 100, loss = 0.6958
t = 200, loss = 0.7500
t = 300, loss = 0.6756
Got 34092 / 45000 correct (75.76)
Starting epoch 6
t = 100, loss = 0.5969
t = 200, loss = 0.8006
t = 300, loss = 0.6283
Got 34806 / 45000 correct (77.35)
Starting epoch 7
t = 100, loss = 0.6309
t = 200, loss = 0.7277
t = 300, loss = 0.5306
Got 35079 / 45000 correct (77.95)
Starting epoch 8
t = 100, loss = 0.6337
t = 200, loss = 0.7113
t = 300, loss = 0.4895
Got 35

t = 300, loss = 0.1094
Got 42805 / 45000 correct (95.12)
Starting epoch 136
t = 100, loss = 0.0990
t = 200, loss = 0.2530
t = 300, loss = 0.1146
Got 42556 / 45000 correct (94.57)
Starting epoch 137
t = 100, loss = 0.0775
t = 200, loss = 0.2253
t = 300, loss = 0.1317
Got 42810 / 45000 correct (95.13)
Starting epoch 138
t = 100, loss = 0.2168
t = 200, loss = 0.1943
t = 300, loss = 0.1722
Got 42560 / 45000 correct (94.58)
Starting epoch 139
t = 100, loss = 0.1029
t = 200, loss = 0.1594
t = 300, loss = 0.0930
Got 42804 / 45000 correct (95.12)
Starting epoch 140
t = 100, loss = 0.1203
t = 200, loss = 0.2417
t = 300, loss = 0.0917
Got 42831 / 45000 correct (95.18)
Starting epoch 141
t = 100, loss = 0.1098
t = 200, loss = 0.1456
t = 300, loss = 0.1030
Got 42707 / 45000 correct (94.90)
Starting epoch 142
t = 100, loss = 0.1601
t = 200, loss = 0.1820
t = 300, loss = 0.1128
Got 42676 / 45000 correct (94.84)
Starting epoch 143
t = 100, loss = 0.1394
t = 200, loss = 0.1286
t = 300, loss = 0.1320
G

In [24]:
outputs_prune_40_outs= pruning_fine_tuning(model_outputs, conv_rate=0.4, 
                                           fc_rate=0.4, epochs=200, 
                                           filename='model outputs prune 40 out',
                                           saving=True, out=True)

Model pruned with rates: 0.4 , 0.4
Got 6410 / 45000 correct (14.24)
Starting epoch 1
t = 100, loss = 1.6958
t = 200, loss = 1.4378
t = 300, loss = 1.4965
Got 19916 / 45000 correct (44.26)
Starting epoch 2
t = 100, loss = 1.2635
t = 200, loss = 1.2476
t = 300, loss = 1.2205
Got 22912 / 45000 correct (50.92)
Starting epoch 3
t = 100, loss = 1.1770
t = 200, loss = 1.1695
t = 300, loss = 1.3422
Got 24684 / 45000 correct (54.85)
Starting epoch 4
t = 100, loss = 1.0757
t = 200, loss = 0.9651
t = 300, loss = 1.1165
Got 25027 / 45000 correct (55.62)
Starting epoch 5
t = 100, loss = 1.0602
t = 200, loss = 1.0428
t = 300, loss = 1.0247
Got 25229 / 45000 correct (56.06)
Starting epoch 6
t = 100, loss = 0.9705
t = 200, loss = 1.0302
t = 300, loss = 1.0234
Got 26293 / 45000 correct (58.43)
Starting epoch 7
t = 100, loss = 0.9623
t = 200, loss = 0.9341
t = 300, loss = 0.9437
Got 25787 / 45000 correct (57.30)
Starting epoch 8
t = 100, loss = 0.8482
t = 200, loss = 0.9859
t = 300, loss = 1.0136
Got 26

t = 300, loss = 0.6510
Got 30709 / 45000 correct (68.24)
Starting epoch 136
t = 100, loss = 0.5887
t = 200, loss = 0.5739
t = 300, loss = 0.6330
Got 30587 / 45000 correct (67.97)
Starting epoch 137
t = 100, loss = 0.5355
t = 200, loss = 0.6349
t = 300, loss = 0.6231
Got 30629 / 45000 correct (68.06)
Starting epoch 138
t = 100, loss = 0.5956
t = 200, loss = 0.6254
t = 300, loss = 0.6585
Got 30561 / 45000 correct (67.91)
Starting epoch 139
t = 100, loss = 0.5410
t = 200, loss = 0.6587
t = 300, loss = 0.6037
Got 30754 / 45000 correct (68.34)
Starting epoch 140
t = 100, loss = 0.5900
t = 200, loss = 0.5859
t = 300, loss = 0.6485
Got 30478 / 45000 correct (67.73)
Starting epoch 141
t = 100, loss = 0.5738
t = 200, loss = 0.6456
t = 300, loss = 0.6048
Got 30373 / 45000 correct (67.50)
Starting epoch 142
t = 100, loss = 0.5864
t = 200, loss = 0.5786
t = 300, loss = 0.5970
Got 30773 / 45000 correct (68.38)
Starting epoch 143
t = 100, loss = 0.6158
t = 200, loss = 0.5540
t = 300, loss = 0.6305
G

In [25]:
weights_prune_40 = pruning_fine_tuning(model_weights, conv_rate=0.4, 
                                       fc_rate=0.4, epochs=200,  
                                       filename='model weights prune 40', 
                                       saving=True)

Model pruned with rates: 0.4 , 0.4
Got 13388 / 45000 correct (29.75)
Starting epoch 1
t = 100, loss = 1.4750
t = 200, loss = 1.3598
t = 300, loss = 1.3187
Got 21969 / 45000 correct (48.82)
Starting epoch 2
t = 100, loss = 1.2203
t = 200, loss = 1.0755
t = 300, loss = 1.0820
Got 27679 / 45000 correct (61.51)
Starting epoch 3
t = 100, loss = 0.9647
t = 200, loss = 0.9764
t = 300, loss = 0.9505
Got 28454 / 45000 correct (63.23)
Starting epoch 4
t = 100, loss = 0.8490
t = 200, loss = 0.9257
t = 300, loss = 0.8499
Got 31546 / 45000 correct (70.10)
Starting epoch 5
t = 100, loss = 0.8307
t = 200, loss = 0.9041
t = 300, loss = 0.9155
Got 30351 / 45000 correct (67.45)
Starting epoch 6
t = 100, loss = 0.7927
t = 200, loss = 0.8487
t = 300, loss = 0.8099
Got 31965 / 45000 correct (71.03)
Starting epoch 7
t = 100, loss = 0.7203
t = 200, loss = 0.7958
t = 300, loss = 0.7174
Got 33869 / 45000 correct (75.26)
Starting epoch 8
t = 100, loss = 0.6569
t = 200, loss = 0.8851
t = 300, loss = 0.6763
Got 3

t = 300, loss = 0.1858
Got 42290 / 45000 correct (93.98)
Starting epoch 136
t = 100, loss = 0.1615
t = 200, loss = 0.3219
t = 300, loss = 0.0622
Got 41889 / 45000 correct (93.09)
Starting epoch 137
t = 100, loss = 0.1839
t = 200, loss = 0.1394
t = 300, loss = 0.1329
Got 41737 / 45000 correct (92.75)
Starting epoch 138
t = 100, loss = 0.2287
t = 200, loss = 0.3071
t = 300, loss = 0.1765
Got 41797 / 45000 correct (92.88)
Starting epoch 139
t = 100, loss = 0.1790
t = 200, loss = 0.2258
t = 300, loss = 0.0996
Got 42155 / 45000 correct (93.68)
Starting epoch 140
t = 100, loss = 0.0904
t = 200, loss = 0.2054
t = 300, loss = 0.1545
Got 42036 / 45000 correct (93.41)
Starting epoch 141
t = 100, loss = 0.1273
t = 200, loss = 0.1545
t = 300, loss = 0.0875
Got 42148 / 45000 correct (93.66)
Starting epoch 142
t = 100, loss = 0.1502
t = 200, loss = 0.3340
t = 300, loss = 0.1497
Got 41945 / 45000 correct (93.21)
Starting epoch 143
t = 100, loss = 0.1289
t = 200, loss = 0.2214
t = 300, loss = 0.1588
G

In [26]:
weights_prune_40_outs= pruning_fine_tuning(model_weights, conv_rate=0.4, 
                                           fc_rate=0.4, epochs=200, 
                                           filename='model weights prune 40 out',
                                           saving=True, out=True)

Model pruned with rates: 0.4 , 0.4
Got 6560 / 45000 correct (14.58)
Starting epoch 1
t = 100, loss = 1.6133
t = 200, loss = 1.4242
t = 300, loss = 1.3983
Got 19648 / 45000 correct (43.66)
Starting epoch 2
t = 100, loss = 1.3596
t = 200, loss = 1.2795
t = 300, loss = 1.2842
Got 22954 / 45000 correct (51.01)
Starting epoch 3
t = 100, loss = 1.2907
t = 200, loss = 1.1881
t = 300, loss = 1.2257
Got 21674 / 45000 correct (48.16)
Starting epoch 4
t = 100, loss = 1.2776
t = 200, loss = 1.1480
t = 300, loss = 1.1247
Got 24737 / 45000 correct (54.97)
Starting epoch 5
t = 100, loss = 1.1595
t = 200, loss = 1.0590
t = 300, loss = 1.1083
Got 24805 / 45000 correct (55.12)
Starting epoch 6
t = 100, loss = 1.0539
t = 200, loss = 1.0859
t = 300, loss = 1.0358
Got 22840 / 45000 correct (50.76)
Starting epoch 7
t = 100, loss = 1.0449
t = 200, loss = 0.9798
t = 300, loss = 1.1115
Got 25675 / 45000 correct (57.06)
Starting epoch 8
t = 100, loss = 0.9672
t = 200, loss = 0.9438
t = 300, loss = 1.1790
Got 24

t = 300, loss = 0.6757
Got 30417 / 45000 correct (67.59)
Starting epoch 136
t = 100, loss = 0.5132
t = 200, loss = 0.6061
t = 300, loss = 0.6159
Got 30371 / 45000 correct (67.49)
Starting epoch 137
t = 100, loss = 0.6006
t = 200, loss = 0.6147
t = 300, loss = 0.6483
Got 30583 / 45000 correct (67.96)
Starting epoch 138
t = 100, loss = 0.5392
t = 200, loss = 0.6463
t = 300, loss = 0.6312
Got 30461 / 45000 correct (67.69)
Starting epoch 139
t = 100, loss = 0.5261
t = 200, loss = 0.6342
t = 300, loss = 0.6356
Got 30329 / 45000 correct (67.40)
Starting epoch 140
t = 100, loss = 0.6041
t = 200, loss = 0.6059
t = 300, loss = 0.6015
Got 30559 / 45000 correct (67.91)
Starting epoch 141
t = 100, loss = 0.5318
t = 200, loss = 0.6014
t = 300, loss = 0.6386
Got 30444 / 45000 correct (67.65)
Starting epoch 142
t = 100, loss = 0.6335
t = 200, loss = 0.6730
t = 300, loss = 0.6385
Got 30325 / 45000 correct (67.39)
Starting epoch 143
t = 100, loss = 0.5145
t = 200, loss = 0.6908
t = 300, loss = 0.6374
G

In [27]:
check_accuracy(outputs_prune_40, loader_test, gpu_dtype)
check_accuracy(outputs_prune_40_outs, loader_test, gpu_dtype)
check_accuracy(weights_prune_40, loader_test, gpu_dtype)
check_accuracy(weights_prune_40_outs, loader_test, gpu_dtype)

Got 8654 / 10000 correct (86.54)
Got 6298 / 10000 correct (62.98)
Got 8527 / 10000 correct (85.27)
Got 6264 / 10000 correct (62.64)


In [28]:
outputs_prune_50 = pruning_fine_tuning(model_outputs, conv_rate=0.5, 
                                       fc_rate=0.5, epochs=200, 
                                       filename='model outputs prune 50', 
                                       saving=True)

Model pruned with rates: 0.5 , 0.5
Got 4480 / 45000 correct (9.96)
Starting epoch 1
t = 100, loss = 1.7184
t = 200, loss = 1.4019
t = 300, loss = 1.2872
Got 19229 / 45000 correct (42.73)
Starting epoch 2
t = 100, loss = 1.4114
t = 200, loss = 1.1499
t = 300, loss = 1.1461
Got 23987 / 45000 correct (53.30)
Starting epoch 3
t = 100, loss = 1.1025
t = 200, loss = 1.0046
t = 300, loss = 0.9807
Got 27466 / 45000 correct (61.04)
Starting epoch 4
t = 100, loss = 0.9971
t = 200, loss = 0.9500
t = 300, loss = 1.0044
Got 27921 / 45000 correct (62.05)
Starting epoch 5
t = 100, loss = 0.8657
t = 200, loss = 0.8848
t = 300, loss = 0.8330
Got 30871 / 45000 correct (68.60)
Starting epoch 6
t = 100, loss = 0.9547
t = 200, loss = 0.8636
t = 300, loss = 0.8333
Got 31767 / 45000 correct (70.59)
Starting epoch 7
t = 100, loss = 0.9240
t = 200, loss = 0.8447
t = 300, loss = 0.7510
Got 32864 / 45000 correct (73.03)
Starting epoch 8
t = 100, loss = 0.8559
t = 200, loss = 0.7812
t = 300, loss = 0.7434
Got 320

t = 300, loss = 0.1818
Got 40480 / 45000 correct (89.96)
Starting epoch 136
t = 100, loss = 0.2291
t = 200, loss = 0.3684
t = 300, loss = 0.1874
Got 40740 / 45000 correct (90.53)
Starting epoch 137
t = 100, loss = 0.2737
t = 200, loss = 0.3459
t = 300, loss = 0.2379
Got 40736 / 45000 correct (90.52)
Starting epoch 138
t = 100, loss = 0.2798
t = 200, loss = 0.2777
t = 300, loss = 0.0742
Got 40636 / 45000 correct (90.30)
Starting epoch 139
t = 100, loss = 0.1647
t = 200, loss = 0.4542
t = 300, loss = 0.1608
Got 40653 / 45000 correct (90.34)
Starting epoch 140
t = 100, loss = 0.2028
t = 200, loss = 0.4441
t = 300, loss = 0.1889
Got 40893 / 45000 correct (90.87)
Starting epoch 141
t = 100, loss = 0.2057
t = 200, loss = 0.4959
t = 300, loss = 0.2391
Got 40460 / 45000 correct (89.91)
Starting epoch 142
t = 100, loss = 0.2106
t = 200, loss = 0.2987
t = 300, loss = 0.1431
Got 41020 / 45000 correct (91.16)
Starting epoch 143
t = 100, loss = 0.2476
t = 200, loss = 0.3747
t = 300, loss = 0.1813
G

In [29]:
outputs_prune_50_outs= pruning_fine_tuning(model_outputs, conv_rate=0.5, 
                                           fc_rate=0.5, epochs=200, 
                                           filename='model outputs prune 50 out',
                                           saving=True, out=True)

Model pruned with rates: 0.5 , 0.5
Got 5884 / 45000 correct (13.08)
Starting epoch 1
t = 100, loss = 1.6704
t = 200, loss = 1.5771
t = 300, loss = 1.5478
Got 18044 / 45000 correct (40.10)
Starting epoch 2
t = 100, loss = 1.5433
t = 200, loss = 1.4244
t = 300, loss = 1.3941
Got 19941 / 45000 correct (44.31)
Starting epoch 3
t = 100, loss = 1.4278
t = 200, loss = 1.3649
t = 300, loss = 1.3749
Got 21848 / 45000 correct (48.55)
Starting epoch 4
t = 100, loss = 1.2997
t = 200, loss = 1.2709
t = 300, loss = 1.3194
Got 22313 / 45000 correct (49.58)
Starting epoch 5
t = 100, loss = 1.2977
t = 200, loss = 1.2714
t = 300, loss = 1.2822
Got 22752 / 45000 correct (50.56)
Starting epoch 6
t = 100, loss = 1.3090
t = 200, loss = 1.2961
t = 300, loss = 1.2119
Got 23197 / 45000 correct (51.55)
Starting epoch 7
t = 100, loss = 1.2164
t = 200, loss = 1.2028
t = 300, loss = 1.1698
Got 23195 / 45000 correct (51.54)
Starting epoch 8
t = 100, loss = 1.2633
t = 200, loss = 1.2057
t = 300, loss = 1.1606
Got 23

t = 300, loss = 0.9113
Got 26297 / 45000 correct (58.44)
Starting epoch 136
t = 100, loss = 0.9241
t = 200, loss = 0.8869
t = 300, loss = 0.9027
Got 26008 / 45000 correct (57.80)
Starting epoch 137
t = 100, loss = 0.8619
t = 200, loss = 0.8657
t = 300, loss = 0.9133
Got 25949 / 45000 correct (57.66)
Starting epoch 138
t = 100, loss = 0.9041
t = 200, loss = 0.8733
t = 300, loss = 0.8471
Got 26110 / 45000 correct (58.02)
Starting epoch 139
t = 100, loss = 0.8983
t = 200, loss = 0.8752
t = 300, loss = 0.8551
Got 26231 / 45000 correct (58.29)
Starting epoch 140
t = 100, loss = 0.8975
t = 200, loss = 0.9122
t = 300, loss = 0.8357
Got 26222 / 45000 correct (58.27)
Starting epoch 141
t = 100, loss = 0.9213
t = 200, loss = 0.8686
t = 300, loss = 0.8689
Got 26040 / 45000 correct (57.87)
Starting epoch 142
t = 100, loss = 0.9695
t = 200, loss = 0.8801
t = 300, loss = 0.9318
Got 26064 / 45000 correct (57.92)
Starting epoch 143
t = 100, loss = 0.8866
t = 200, loss = 0.8730
t = 300, loss = 0.8730
G

In [30]:
weights_prune_50 = pruning_fine_tuning(model_weights, conv_rate=0.5, 
                                       fc_rate=0.5, epochs=200,  
                                       filename='model weights prune 50', 
                                       saving=True)

Model pruned with rates: 0.5 , 0.5
Got 8390 / 45000 correct (18.64)
Starting epoch 1
t = 100, loss = 1.9103
t = 200, loss = 1.6164
t = 300, loss = 1.5303
Got 19070 / 45000 correct (42.38)
Starting epoch 2
t = 100, loss = 1.3479
t = 200, loss = 1.2166
t = 300, loss = 1.2116
Got 22973 / 45000 correct (51.05)
Starting epoch 3
t = 100, loss = 1.2190
t = 200, loss = 1.0701
t = 300, loss = 1.0476
Got 21188 / 45000 correct (47.08)
Starting epoch 4
t = 100, loss = 1.1374
t = 200, loss = 0.9490
t = 300, loss = 1.0577
Got 24952 / 45000 correct (55.45)
Starting epoch 5
t = 100, loss = 0.8931
t = 200, loss = 0.9796
t = 300, loss = 0.9598
Got 28416 / 45000 correct (63.15)
Starting epoch 6
t = 100, loss = 0.9523
t = 200, loss = 0.9604
t = 300, loss = 0.9040
Got 29403 / 45000 correct (65.34)
Starting epoch 7
t = 100, loss = 0.9232
t = 200, loss = 0.8977
t = 300, loss = 0.8505
Got 30690 / 45000 correct (68.20)
Starting epoch 8
t = 100, loss = 0.9474
t = 200, loss = 0.8041
t = 300, loss = 0.7720
Got 32

t = 300, loss = 0.2617
Got 40444 / 45000 correct (89.88)
Starting epoch 136
t = 100, loss = 0.3166
t = 200, loss = 0.3931
t = 300, loss = 0.2284
Got 39962 / 45000 correct (88.80)
Starting epoch 137
t = 100, loss = 0.1866
t = 200, loss = 0.4143
t = 300, loss = 0.2824
Got 39927 / 45000 correct (88.73)
Starting epoch 138
t = 100, loss = 0.2027
t = 200, loss = 0.3884
t = 300, loss = 0.2930
Got 40732 / 45000 correct (90.52)
Starting epoch 139
t = 100, loss = 0.2709
t = 200, loss = 0.2878
t = 300, loss = 0.3317
Got 40531 / 45000 correct (90.07)
Starting epoch 140
t = 100, loss = 0.2593
t = 200, loss = 0.2891
t = 300, loss = 0.2018
Got 40132 / 45000 correct (89.18)
Starting epoch 141
t = 100, loss = 0.2773
t = 200, loss = 0.4150
t = 300, loss = 0.2890
Got 40425 / 45000 correct (89.83)
Starting epoch 142
t = 100, loss = 0.2811
t = 200, loss = 0.2879
t = 300, loss = 0.2000
Got 40576 / 45000 correct (90.17)
Starting epoch 143
t = 100, loss = 0.2035
t = 200, loss = 0.2869
t = 300, loss = 0.2784
G

In [31]:
weights_prune_50_outs= pruning_fine_tuning(model_weights, conv_rate=0.5, 
                                           fc_rate=0.5, epochs=200, 
                                           filename='model weights prune 50 out',
                                           saving=True, out=True)

Model pruned with rates: 0.5 , 0.5
Got 4468 / 45000 correct (9.93)
Starting epoch 1
t = 100, loss = 1.8247
t = 200, loss = 1.5590
t = 300, loss = 1.4374
Got 17809 / 45000 correct (39.58)
Starting epoch 2
t = 100, loss = 1.4814
t = 200, loss = 1.4009
t = 300, loss = 1.4263
Got 19933 / 45000 correct (44.30)
Starting epoch 3
t = 100, loss = 1.4935
t = 200, loss = 1.3142
t = 300, loss = 1.2361
Got 18844 / 45000 correct (41.88)
Starting epoch 4
t = 100, loss = 1.4232
t = 200, loss = 1.3295
t = 300, loss = 1.3482
Got 21257 / 45000 correct (47.24)
Starting epoch 5
t = 100, loss = 1.4063
t = 200, loss = 1.2718
t = 300, loss = 1.2736
Got 21778 / 45000 correct (48.40)
Starting epoch 6
t = 100, loss = 1.4102
t = 200, loss = 1.2348
t = 300, loss = 1.2167
Got 21974 / 45000 correct (48.83)
Starting epoch 7
t = 100, loss = 1.3340
t = 200, loss = 1.2458
t = 300, loss = 1.2293
Got 23119 / 45000 correct (51.38)
Starting epoch 8
t = 100, loss = 1.2683
t = 200, loss = 1.1852
t = 300, loss = 1.2242
Got 221

t = 300, loss = 0.8844
Got 26026 / 45000 correct (57.84)
Starting epoch 136
t = 100, loss = 0.9068
t = 200, loss = 0.8852
t = 300, loss = 0.9094
Got 26082 / 45000 correct (57.96)
Starting epoch 137
t = 100, loss = 0.8965
t = 200, loss = 0.8708
t = 300, loss = 0.8754
Got 26057 / 45000 correct (57.90)
Starting epoch 138
t = 100, loss = 0.9206
t = 200, loss = 0.8875
t = 300, loss = 0.9048
Got 25985 / 45000 correct (57.74)
Starting epoch 139
t = 100, loss = 0.9285
t = 200, loss = 0.8440
t = 300, loss = 0.8578
Got 25897 / 45000 correct (57.55)
Starting epoch 140
t = 100, loss = 0.9619
t = 200, loss = 0.9122
t = 300, loss = 0.8438
Got 26078 / 45000 correct (57.95)
Starting epoch 141
t = 100, loss = 0.9015
t = 200, loss = 0.9071
t = 300, loss = 0.8550
Got 26080 / 45000 correct (57.96)
Starting epoch 142
t = 100, loss = 0.9225
t = 200, loss = 0.8342
t = 300, loss = 0.8446
Got 25965 / 45000 correct (57.70)
Starting epoch 143
t = 100, loss = 0.9317
t = 200, loss = 0.8715
t = 300, loss = 0.8432
G

In [32]:
check_accuracy(outputs_prune_50, loader_test, gpu_dtype)
check_accuracy(outputs_prune_50_outs, loader_test, gpu_dtype)
check_accuracy(weights_prune_50, loader_test, gpu_dtype)
check_accuracy(weights_prune_50_outs, loader_test, gpu_dtype)

Got 8321 / 10000 correct (83.21)
Got 5366 / 10000 correct (53.66)
Got 8169 / 10000 correct (81.69)
Got 5461 / 10000 correct (54.61)


In [33]:
outputs_prune_60 = pruning_fine_tuning(model_outputs, conv_rate=0.6, 
                                       fc_rate=0.6, epochs=200, 
                                       filename='model outputs prune 60', 
                                       saving=True)

Model pruned with rates: 0.6 , 0.6
Got 4471 / 45000 correct (9.94)
Starting epoch 1
t = 100, loss = 1.9287
t = 200, loss = 1.8985
t = 300, loss = 1.6912
Got 13778 / 45000 correct (30.62)
Starting epoch 2
t = 100, loss = 1.3987
t = 200, loss = 1.2705
t = 300, loss = 1.3445
Got 23521 / 45000 correct (52.27)
Starting epoch 3
t = 100, loss = 1.4009
t = 200, loss = 1.1412
t = 300, loss = 1.1022
Got 24011 / 45000 correct (53.36)
Starting epoch 4
t = 100, loss = 1.0735
t = 200, loss = 1.1360
t = 300, loss = 1.0981
Got 21898 / 45000 correct (48.66)
Starting epoch 5
t = 100, loss = 1.1451
t = 200, loss = 1.0017
t = 300, loss = 0.9756
Got 29172 / 45000 correct (64.83)
Starting epoch 6
t = 100, loss = 1.0638
t = 200, loss = 1.0309
t = 300, loss = 0.8998
Got 29528 / 45000 correct (65.62)
Starting epoch 7
t = 100, loss = 1.0158
t = 200, loss = 0.9635
t = 300, loss = 0.8449
Got 30324 / 45000 correct (67.39)
Starting epoch 8
t = 100, loss = 0.8427
t = 200, loss = 0.9746
t = 300, loss = 0.9110
Got 315

t = 300, loss = 0.1443
Got 39754 / 45000 correct (88.34)
Starting epoch 136
t = 100, loss = 0.3114
t = 200, loss = 0.3417
t = 300, loss = 0.2188
Got 39706 / 45000 correct (88.24)
Starting epoch 137
t = 100, loss = 0.3224
t = 200, loss = 0.3437
t = 300, loss = 0.2549
Got 39946 / 45000 correct (88.77)
Starting epoch 138
t = 100, loss = 0.2434
t = 200, loss = 0.4049
t = 300, loss = 0.2406
Got 39657 / 45000 correct (88.13)
Starting epoch 139
t = 100, loss = 0.2368
t = 200, loss = 0.4466
t = 300, loss = 0.3252
Got 39744 / 45000 correct (88.32)
Starting epoch 140
t = 100, loss = 0.3110
t = 200, loss = 0.4105
t = 300, loss = 0.1810
Got 39908 / 45000 correct (88.68)
Starting epoch 141
t = 100, loss = 0.2479
t = 200, loss = 0.3818
t = 300, loss = 0.2459
Got 39701 / 45000 correct (88.22)
Starting epoch 142
t = 100, loss = 0.2098
t = 200, loss = 0.4214
t = 300, loss = 0.2305
Got 39404 / 45000 correct (87.56)
Starting epoch 143
t = 100, loss = 0.2184
t = 200, loss = 0.4335
t = 300, loss = 0.2920
G

In [34]:
outputs_prune_60_outs= pruning_fine_tuning(model_outputs, conv_rate=0.6, 
                                           fc_rate=0.6, epochs=200, 
                                           filename='model outputs prune 60 out',
                                           saving=True, out=True)

Model pruned with rates: 0.6 , 0.6
Got 3765 / 45000 correct (8.37)
Starting epoch 1
t = 100, loss = 2.0622
t = 200, loss = 1.8560
t = 300, loss = 1.8197
Got 12757 / 45000 correct (28.35)
Starting epoch 2
t = 100, loss = 1.7941
t = 200, loss = 1.6694
t = 300, loss = 1.7150
Got 15190 / 45000 correct (33.76)
Starting epoch 3
t = 100, loss = 1.8398
t = 200, loss = 1.6247
t = 300, loss = 1.6990
Got 14610 / 45000 correct (32.47)
Starting epoch 4
t = 100, loss = 1.7626
t = 200, loss = 1.5310
t = 300, loss = 1.6150
Got 15528 / 45000 correct (34.51)
Starting epoch 5
t = 100, loss = 1.6592
t = 200, loss = 1.5271
t = 300, loss = 1.6111
Got 17752 / 45000 correct (39.45)
Starting epoch 6
t = 100, loss = 1.7103
t = 200, loss = 1.5194
t = 300, loss = 1.6823
Got 17487 / 45000 correct (38.86)
Starting epoch 7
t = 100, loss = 1.6272
t = 200, loss = 1.4616
t = 300, loss = 1.6030
Got 17110 / 45000 correct (38.02)
Starting epoch 8
t = 100, loss = 1.6120
t = 200, loss = 1.3926
t = 300, loss = 1.5837
Got 174

t = 300, loss = 1.1572
Got 21311 / 45000 correct (47.36)
Starting epoch 136
t = 100, loss = 1.2004
t = 200, loss = 1.1558
t = 300, loss = 1.1716
Got 21150 / 45000 correct (47.00)
Starting epoch 137
t = 100, loss = 1.2772
t = 200, loss = 1.0948
t = 300, loss = 1.1988
Got 21473 / 45000 correct (47.72)
Starting epoch 138
t = 100, loss = 1.2722
t = 200, loss = 1.1490
t = 300, loss = 1.1660
Got 21390 / 45000 correct (47.53)
Starting epoch 139
t = 100, loss = 1.2700
t = 200, loss = 1.1687
t = 300, loss = 1.1904
Got 21415 / 45000 correct (47.59)
Starting epoch 140
t = 100, loss = 1.2841
t = 200, loss = 1.1814
t = 300, loss = 1.2294
Got 21440 / 45000 correct (47.64)
Starting epoch 141
t = 100, loss = 1.2398
t = 200, loss = 1.1201
t = 300, loss = 1.2051
Got 21278 / 45000 correct (47.28)
Starting epoch 142
t = 100, loss = 1.2639
t = 200, loss = 1.2622
t = 300, loss = 1.1519
Got 21324 / 45000 correct (47.39)
Starting epoch 143
t = 100, loss = 1.2249
t = 200, loss = 1.1687
t = 300, loss = 1.1562
G

In [35]:
weights_prune_60 = pruning_fine_tuning(model_weights, conv_rate=0.6, 
                                       fc_rate=0.6, epochs=200,  
                                       filename='model weights prune 60', 
                                       saving=True)

Model pruned with rates: 0.6 , 0.6
Got 4671 / 45000 correct (10.38)
Starting epoch 1
t = 100, loss = 2.2947
t = 200, loss = 1.9732
t = 300, loss = 1.9033
Got 14298 / 45000 correct (31.77)
Starting epoch 2
t = 100, loss = 1.6532
t = 200, loss = 1.5995
t = 300, loss = 1.5760
Got 17648 / 45000 correct (39.22)
Starting epoch 3
t = 100, loss = 1.3293
t = 200, loss = 1.2369
t = 300, loss = 1.3145
Got 20939 / 45000 correct (46.53)
Starting epoch 4
t = 100, loss = 1.2776
t = 200, loss = 1.1399
t = 300, loss = 1.1316
Got 25149 / 45000 correct (55.89)
Starting epoch 5
t = 100, loss = 1.1969
t = 200, loss = 1.0314
t = 300, loss = 1.0857
Got 27076 / 45000 correct (60.17)
Starting epoch 6
t = 100, loss = 1.0015
t = 200, loss = 0.9672
t = 300, loss = 0.9935
Got 27662 / 45000 correct (61.47)
Starting epoch 7
t = 100, loss = 0.9770
t = 200, loss = 0.9074
t = 300, loss = 0.9443
Got 29970 / 45000 correct (66.60)
Starting epoch 8
t = 100, loss = 1.0323
t = 200, loss = 0.9185
t = 300, loss = 0.9169
Got 29

t = 300, loss = 0.1990
Got 39331 / 45000 correct (87.40)
Starting epoch 136
t = 100, loss = 0.3867
t = 200, loss = 0.4833
t = 300, loss = 0.2703
Got 39418 / 45000 correct (87.60)
Starting epoch 137
t = 100, loss = 0.3741
t = 200, loss = 0.4602
t = 300, loss = 0.3050
Got 38960 / 45000 correct (86.58)
Starting epoch 138
t = 100, loss = 0.4025
t = 200, loss = 0.3714
t = 300, loss = 0.2803
Got 39501 / 45000 correct (87.78)
Starting epoch 139
t = 100, loss = 0.3239
t = 200, loss = 0.4850
t = 300, loss = 0.2234
Got 38890 / 45000 correct (86.42)
Starting epoch 140
t = 100, loss = 0.2840
t = 200, loss = 0.4669
t = 300, loss = 0.2457
Got 39086 / 45000 correct (86.86)
Starting epoch 141
t = 100, loss = 0.3857
t = 200, loss = 0.4640
t = 300, loss = 0.2554
Got 39382 / 45000 correct (87.52)
Starting epoch 142
t = 100, loss = 0.2842
t = 200, loss = 0.4005
t = 300, loss = 0.2328
Got 39268 / 45000 correct (87.26)
Starting epoch 143
t = 100, loss = 0.3456
t = 200, loss = 0.3028
t = 300, loss = 0.2382
G

In [36]:
weights_prune_60_outs= pruning_fine_tuning(model_weights, conv_rate=0.6, 
                                           fc_rate=0.6, epochs=200, 
                                           filename='model weights prune 60 out',
                                           saving=True, out=True)

Model pruned with rates: 0.6 , 0.6
Got 4468 / 45000 correct (9.93)
Starting epoch 1
t = 100, loss = 2.0906
t = 200, loss = 1.9839
t = 300, loss = 1.8607
Got 10044 / 45000 correct (22.32)
Starting epoch 2
t = 100, loss = 1.9067
t = 200, loss = 1.7824
t = 300, loss = 1.7467
Got 11290 / 45000 correct (25.09)
Starting epoch 3
t = 100, loss = 1.7721
t = 200, loss = 1.6632
t = 300, loss = 1.6838
Got 14919 / 45000 correct (33.15)
Starting epoch 4
t = 100, loss = 1.8206
t = 200, loss = 1.6118
t = 300, loss = 1.6409
Got 15118 / 45000 correct (33.60)
Starting epoch 5
t = 100, loss = 1.6835
t = 200, loss = 1.6035
t = 300, loss = 1.7439
Got 15924 / 45000 correct (35.39)
Starting epoch 6
t = 100, loss = 1.6860
t = 200, loss = 1.5606
t = 300, loss = 1.6608
Got 16000 / 45000 correct (35.56)
Starting epoch 7
t = 100, loss = 1.7344
t = 200, loss = 1.5709
t = 300, loss = 1.6806
Got 16582 / 45000 correct (36.85)
Starting epoch 8
t = 100, loss = 1.6664
t = 200, loss = 1.6146
t = 300, loss = 1.6781
Got 163

t = 300, loss = 1.3121
Got 21131 / 45000 correct (46.96)
Starting epoch 136
t = 100, loss = 1.3394
t = 200, loss = 1.1776
t = 300, loss = 1.3069
Got 20834 / 45000 correct (46.30)
Starting epoch 137
t = 100, loss = 1.3645
t = 200, loss = 1.1933
t = 300, loss = 1.2682
Got 21024 / 45000 correct (46.72)
Starting epoch 138
t = 100, loss = 1.3393
t = 200, loss = 1.1986
t = 300, loss = 1.2694
Got 21146 / 45000 correct (46.99)
Starting epoch 139
t = 100, loss = 1.3485
t = 200, loss = 1.1951
t = 300, loss = 1.2517
Got 21051 / 45000 correct (46.78)
Starting epoch 140
t = 100, loss = 1.2683
t = 200, loss = 1.1830
t = 300, loss = 1.2506
Got 21112 / 45000 correct (46.92)
Starting epoch 141
t = 100, loss = 1.3726
t = 200, loss = 1.1781
t = 300, loss = 1.2705
Got 20777 / 45000 correct (46.17)
Starting epoch 142
t = 100, loss = 1.3550
t = 200, loss = 1.2452
t = 300, loss = 1.2327
Got 21091 / 45000 correct (46.87)
Starting epoch 143
t = 100, loss = 1.3718
t = 200, loss = 1.2082
t = 300, loss = 1.3255
G

In [37]:
check_accuracy(outputs_prune_60, loader_test, gpu_dtype)
check_accuracy(outputs_prune_60_outs, loader_test, gpu_dtype)
check_accuracy(weights_prune_60, loader_test, gpu_dtype)
check_accuracy(weights_prune_60_outs, loader_test, gpu_dtype)

Got 8248 / 10000 correct (82.48)
Got 4434 / 10000 correct (44.34)
Got 8231 / 10000 correct (82.31)
Got 4405 / 10000 correct (44.05)


In [39]:
outputs_prune_70 = pruning_fine_tuning(model_outputs, conv_rate=0.7, 
                                       fc_rate=0.7, epochs=200, 
                                       filename='model outputs prune 70', 
                                       saving=True)

Model pruned with rates: 0.7 , 0.7
Got 4487 / 45000 correct (9.97)
Starting epoch 1
t = 100, loss = 1.7710
t = 200, loss = 1.7696
t = 300, loss = 1.4870
Got 18148 / 45000 correct (40.33)
Starting epoch 2
t = 100, loss = 1.3526
t = 200, loss = 1.2535
t = 300, loss = 1.3012
Got 20955 / 45000 correct (46.57)
Starting epoch 3
t = 100, loss = 1.3441
t = 200, loss = 1.1509
t = 300, loss = 1.2356
Got 21531 / 45000 correct (47.85)
Starting epoch 4
t = 100, loss = 1.1978
t = 200, loss = 0.9949
t = 300, loss = 1.1356
Got 24598 / 45000 correct (54.66)
Starting epoch 5
t = 100, loss = 1.0685
t = 200, loss = 1.1202
t = 300, loss = 0.9861
Got 28629 / 45000 correct (63.62)
Starting epoch 6
t = 100, loss = 1.0199
t = 200, loss = 1.0093
t = 300, loss = 0.9872
Got 28277 / 45000 correct (62.84)
Starting epoch 7
t = 100, loss = 1.0079
t = 200, loss = 0.9207
t = 300, loss = 0.9311
Got 29598 / 45000 correct (65.77)
Starting epoch 8
t = 100, loss = 0.9401
t = 200, loss = 0.9502
t = 300, loss = 0.7833
Got 298

t = 300, loss = 0.4472
Got 38146 / 45000 correct (84.77)
Starting epoch 136
t = 100, loss = 0.4656
t = 200, loss = 0.5256
t = 300, loss = 0.3582
Got 37809 / 45000 correct (84.02)
Starting epoch 137
t = 100, loss = 0.4613
t = 200, loss = 0.5221
t = 300, loss = 0.4865
Got 37949 / 45000 correct (84.33)
Starting epoch 138
t = 100, loss = 0.4072
t = 200, loss = 0.5548
t = 300, loss = 0.4240
Got 37280 / 45000 correct (82.84)
Starting epoch 139
t = 100, loss = 0.4848
t = 200, loss = 0.5389
t = 300, loss = 0.3101
Got 38211 / 45000 correct (84.91)
Starting epoch 140
t = 100, loss = 0.4243
t = 200, loss = 0.6113
t = 300, loss = 0.3630
Got 37829 / 45000 correct (84.06)
Starting epoch 141
t = 100, loss = 0.5289
t = 200, loss = 0.5841
t = 300, loss = 0.4261
Got 38204 / 45000 correct (84.90)
Starting epoch 142
t = 100, loss = 0.4086
t = 200, loss = 0.5562
t = 300, loss = 0.3955
Got 37442 / 45000 correct (83.20)
Starting epoch 143
t = 100, loss = 0.5302
t = 200, loss = 0.6238
t = 300, loss = 0.4139
G

In [40]:
outputs_prune_70_outs= pruning_fine_tuning(model_outputs, conv_rate=0.7, 
                                           fc_rate=0.7, epochs=200, 
                                           filename='model outputs prune 70 out',
                                           saving=True, out=True)

Model pruned with rates: 0.7 , 0.7
Got 5116 / 45000 correct (11.37)
Starting epoch 1
t = 100, loss = 2.0821
t = 200, loss = 1.9939
t = 300, loss = 2.0753
Got 11174 / 45000 correct (24.83)
Starting epoch 2
t = 100, loss = 2.0415
t = 200, loss = 1.9406
t = 300, loss = 1.9549
Got 11842 / 45000 correct (26.32)
Starting epoch 3
t = 100, loss = 1.9003
t = 200, loss = 1.8346
t = 300, loss = 1.9423
Got 13710 / 45000 correct (30.47)
Starting epoch 4
t = 100, loss = 1.8969
t = 200, loss = 1.8886
t = 300, loss = 1.9615
Got 13345 / 45000 correct (29.66)
Starting epoch 5
t = 100, loss = 1.8574
t = 200, loss = 1.7675
t = 300, loss = 2.0197
Got 13723 / 45000 correct (30.50)
Starting epoch 6
t = 100, loss = 1.9351
t = 200, loss = 1.8090
t = 300, loss = 1.8927
Got 14266 / 45000 correct (31.70)
Starting epoch 7
t = 100, loss = 1.8451
t = 200, loss = 1.7583
t = 300, loss = 1.8919
Got 14099 / 45000 correct (31.33)
Starting epoch 8
t = 100, loss = 1.7835
t = 200, loss = 1.7391
t = 300, loss = 1.8413
Got 14

t = 300, loss = 1.5543
Got 16755 / 45000 correct (37.23)
Starting epoch 136
t = 100, loss = 1.5425
t = 200, loss = 1.4935
t = 300, loss = 1.5530
Got 16911 / 45000 correct (37.58)
Starting epoch 137
t = 100, loss = 1.6161
t = 200, loss = 1.5500
t = 300, loss = 1.5657
Got 16969 / 45000 correct (37.71)
Starting epoch 138
t = 100, loss = 1.5964
t = 200, loss = 1.5097
t = 300, loss = 1.6239
Got 16947 / 45000 correct (37.66)
Starting epoch 139
t = 100, loss = 1.5874
t = 200, loss = 1.4427
t = 300, loss = 1.5635
Got 16899 / 45000 correct (37.55)
Starting epoch 140
t = 100, loss = 1.6426
t = 200, loss = 1.5165
t = 300, loss = 1.5790
Got 16933 / 45000 correct (37.63)
Starting epoch 141
t = 100, loss = 1.6083
t = 200, loss = 1.4915
t = 300, loss = 1.5632
Got 16956 / 45000 correct (37.68)
Starting epoch 142
t = 100, loss = 1.5703
t = 200, loss = 1.4723
t = 300, loss = 1.5677
Got 16577 / 45000 correct (36.84)
Starting epoch 143
t = 100, loss = 1.5096
t = 200, loss = 1.4725
t = 300, loss = 1.5236
G

In [41]:
weights_prune_70 = pruning_fine_tuning(model_weights, conv_rate=0.7, 
                                       fc_rate=0.7, epochs=200,  
                                       filename='model weights prune 70', 
                                       saving=True)

Model pruned with rates: 0.7 , 0.7
Got 4520 / 45000 correct (10.04)
Starting epoch 1
t = 100, loss = 1.9918
t = 200, loss = 1.6982
t = 300, loss = 1.5852
Got 16255 / 45000 correct (36.12)
Starting epoch 2
t = 100, loss = 1.4312
t = 200, loss = 1.4312
t = 300, loss = 1.4738
Got 20883 / 45000 correct (46.41)
Starting epoch 3
t = 100, loss = 1.3174
t = 200, loss = 1.1027
t = 300, loss = 1.3631
Got 22334 / 45000 correct (49.63)
Starting epoch 4
t = 100, loss = 1.2344
t = 200, loss = 1.0611
t = 300, loss = 1.1338
Got 25425 / 45000 correct (56.50)
Starting epoch 5
t = 100, loss = 1.1880
t = 200, loss = 0.9806
t = 300, loss = 1.0863
Got 26832 / 45000 correct (59.63)
Starting epoch 6
t = 100, loss = 1.0124
t = 200, loss = 1.0725
t = 300, loss = 1.0174
Got 27485 / 45000 correct (61.08)
Starting epoch 7
t = 100, loss = 1.0098
t = 200, loss = 1.0120
t = 300, loss = 0.9007
Got 29283 / 45000 correct (65.07)
Starting epoch 8
t = 100, loss = 0.9275
t = 200, loss = 0.9378
t = 300, loss = 0.9468
Got 29

t = 300, loss = 0.4126
Got 37307 / 45000 correct (82.90)
Starting epoch 136
t = 100, loss = 0.4931
t = 200, loss = 0.6359
t = 300, loss = 0.4534
Got 37346 / 45000 correct (82.99)
Starting epoch 137
t = 100, loss = 0.4141
t = 200, loss = 0.6251
t = 300, loss = 0.5133
Got 37424 / 45000 correct (83.16)
Starting epoch 138
t = 100, loss = 0.4941
t = 200, loss = 0.6431
t = 300, loss = 0.4172
Got 37056 / 45000 correct (82.35)
Starting epoch 139
t = 100, loss = 0.4980
t = 200, loss = 0.6877
t = 300, loss = 0.5071
Got 37641 / 45000 correct (83.65)
Starting epoch 140
t = 100, loss = 0.4934
t = 200, loss = 0.6444
t = 300, loss = 0.4554
Got 37228 / 45000 correct (82.73)
Starting epoch 141
t = 100, loss = 0.4043
t = 200, loss = 0.5910
t = 300, loss = 0.4497
Got 37459 / 45000 correct (83.24)
Starting epoch 142
t = 100, loss = 0.4847
t = 200, loss = 0.6312
t = 300, loss = 0.4811
Got 36650 / 45000 correct (81.44)
Starting epoch 143
t = 100, loss = 0.4446
t = 200, loss = 0.5972
t = 300, loss = 0.4072
G

In [42]:
weights_prune_70_outs= pruning_fine_tuning(model_weights, conv_rate=0.7, 
                                           fc_rate=0.7, epochs=200, 
                                           filename='model weights prune 70 out',
                                           saving=True, out=True)

Model pruned with rates: 0.7 , 0.7
Got 4529 / 45000 correct (10.06)
Starting epoch 1
t = 100, loss = 2.3235
t = 200, loss = 2.2417
t = 300, loss = 2.2666
Got 5574 / 45000 correct (12.39)
Starting epoch 2
t = 100, loss = 2.1413
t = 200, loss = 2.0929
t = 300, loss = 2.1671
Got 9704 / 45000 correct (21.56)
Starting epoch 3
t = 100, loss = 2.1082
t = 200, loss = 1.9434
t = 300, loss = 2.0689
Got 10550 / 45000 correct (23.44)
Starting epoch 4
t = 100, loss = 2.0287
t = 200, loss = 1.9567
t = 300, loss = 2.1025
Got 12452 / 45000 correct (27.67)
Starting epoch 5
t = 100, loss = 2.0442
t = 200, loss = 1.8893
t = 300, loss = 2.0235
Got 12355 / 45000 correct (27.46)
Starting epoch 6
t = 100, loss = 1.9449
t = 200, loss = 1.8246
t = 300, loss = 1.9660
Got 13148 / 45000 correct (29.22)
Starting epoch 7
t = 100, loss = 1.9348
t = 200, loss = 1.9001
t = 300, loss = 1.9506
Got 13750 / 45000 correct (30.56)
Starting epoch 8
t = 100, loss = 1.9833
t = 200, loss = 1.9037
t = 300, loss = 1.9728
Got 1325

t = 300, loss = 1.6182
Got 16788 / 45000 correct (37.31)
Starting epoch 136
t = 100, loss = 1.5476
t = 200, loss = 1.5940
t = 300, loss = 1.6560
Got 16754 / 45000 correct (37.23)
Starting epoch 137
t = 100, loss = 1.5335
t = 200, loss = 1.5020
t = 300, loss = 1.6482
Got 16733 / 45000 correct (37.18)
Starting epoch 138
t = 100, loss = 1.5444
t = 200, loss = 1.5000
t = 300, loss = 1.5960
Got 16851 / 45000 correct (37.45)
Starting epoch 139
t = 100, loss = 1.5682
t = 200, loss = 1.4797
t = 300, loss = 1.6394
Got 16745 / 45000 correct (37.21)
Starting epoch 140
t = 100, loss = 1.5720
t = 200, loss = 1.4817
t = 300, loss = 1.5325
Got 16814 / 45000 correct (37.36)
Starting epoch 141
t = 100, loss = 1.6411
t = 200, loss = 1.4893
t = 300, loss = 1.6070
Got 16822 / 45000 correct (37.38)
Starting epoch 142
t = 100, loss = 1.5485
t = 200, loss = 1.4692
t = 300, loss = 1.6179
Got 16699 / 45000 correct (37.11)
Starting epoch 143
t = 100, loss = 1.5308
t = 200, loss = 1.5067
t = 300, loss = 1.6339
G

In [44]:
check_accuracy(outputs_prune_70, loader_test, gpu_dtype)
check_accuracy(outputs_prune_70_outs, loader_test, gpu_dtype)
check_accuracy(weights_prune_70, loader_test, gpu_dtype)
check_accuracy(weights_prune_70_outs, loader_test, gpu_dtype)

Got 7984 / 10000 correct (79.84)
Got 3504 / 10000 correct (35.04)
Got 7856 / 10000 correct (78.56)
Got 3533 / 10000 correct (35.33)


In [46]:
 print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(outputs_prune_70.layers2[0].conv1.weight == 0))
        / float(outputs_prune_70.layers2[0].conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(outputs_prune_70.layers1[0].conv1.weight == 0))
        / float(outputs_prune_70.layers1[0].conv1.weight.nelement())
    )
)

Sparsity in conv2.weight: 68.75%
Sparsity in conv1.weight: 68.75%


In [6]:
weights_prune_60 = torch.load('model weights prune 60')

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'model.ResBlock' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [22]:
for name, module in weights_prune_60.named_modules():
    print(name, module)

 ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (layers1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout2d(p=0.67, inplace=True)
      (relu2): ReLU(inplace=True)
    )
    (1): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), st

)
layers1.0 ResBlock(
  (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout2d(p=0.67, inplace=True)
  (relu2): ReLU(inplace=True)
)
layers1.0.conv1 Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layers1.0.bn1 BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layers1.0.relu1 ReLU(inplace=True)
layers1.0.conv2 Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layers1.0.bn2 BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layers1.0.dropout Dropout2d(p=0.67, inplace=True)
layers1.0.relu2 ReLU(inplace=True)
layers1.1 ResBlock(
  (conv1): Conv2d(16, 16, kernel_size=(3, 3), s

)
layers2.1.conv1 Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layers2.1.bn1 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layers2.1.relu1 ReLU(inplace=True)
layers2.1.conv2 Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layers2.1.bn2 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layers2.1.relu2 ReLU(inplace=True)
layers2.2 ResBlock(
  (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
)
layers2.2.conv1 Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layers2.2.bn1 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_runnin

layers3.2.relu2 ReLU(inplace=True)
layers3.3 ResBlock(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
)
layers3.3.conv1 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layers3.3.bn1 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layers3.3.relu1 ReLU(inplace=True)
layers3.3.conv2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layers3.3.bn2 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layers3.3.relu2 ReLU(inplace=True)
layers3.4 ResBlock(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-

In [21]:
for name, parameter in weights_prune_60.named_parameters():
        print('grad', name, parameter.numel())

grad conv1.bias 16
grad conv1.weight_orig 432
grad norm1.weight 16
grad norm1.bias 16
grad layers1.0.conv1.bias 16
grad layers1.0.conv1.weight_orig 2304
grad layers1.0.bn1.weight 16
grad layers1.0.bn1.bias 16
grad layers1.0.conv2.bias 16
grad layers1.0.conv2.weight_orig 2304
grad layers1.0.bn2.weight 16
grad layers1.0.bn2.bias 16
grad layers1.1.conv1.bias 16
grad layers1.1.conv1.weight_orig 2304
grad layers1.1.bn1.weight 16
grad layers1.1.bn1.bias 16
grad layers1.1.conv2.bias 16
grad layers1.1.conv2.weight_orig 2304
grad layers1.1.bn2.weight 16
grad layers1.1.bn2.bias 16
grad layers1.2.conv1.bias 16
grad layers1.2.conv1.weight_orig 2304
grad layers1.2.bn1.weight 16
grad layers1.2.bn1.bias 16
grad layers1.2.conv2.bias 16
grad layers1.2.conv2.weight_orig 2304
grad layers1.2.bn2.weight 16
grad layers1.2.bn2.bias 16
grad layers1.3.conv1.bias 16
grad layers1.3.conv1.weight_orig 2304
grad layers1.3.bn1.weight 16
grad layers1.3.bn1.bias 16
grad layers1.3.conv2.bias 16
grad layers1.3.conv2.wei

In [24]:
weights_prune_60.linear.weight.shape

torch.Size([10, 64])

In [25]:
weights_prune_60.conv1.weight.shape

torch.Size([16, 3, 3, 3])